In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.stats as stats
import pandas as pd
import numpy as np
import os 
import sklearn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
train_file = pd.read_csv('iot-datasets/train.csv')
test_file = pd.read_csv('iot-datasets/test.csv')
severity_type = pd.read_csv('iot-datasets/severity_type.csv')
event_type = pd.read_csv('iot-datasets/event_type.csv')
log_feature = pd.read_csv('iot-datasets/log_feature.csv')
resource_type = pd.read_csv('iot-datasets/resource_type.csv')

In [3]:
df = train_file.merge(severity_type, how = 'left', left_on='id', right_on='id')
df = df.merge(resource_type, how = 'left', left_on='id', right_on='id')
df = df.merge(log_feature, how = 'left', left_on='id', right_on='id')
df = df.merge(event_type, how = 'left', left_on='id', right_on='id')

In [4]:
df.head()

,id,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 34
1,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 35
2,14121,location 118,1,severity_type 2,resource_type 2,feature 232,19,event_type 34
3,14121,location 118,1,severity_type 2,resource_type 2,feature 232,19,event_type 35
4,9320,location 91,0,severity_type 2,resource_type 2,feature 315,200,event_type 34


In [5]:
df.isnull().sum()

id                0
location          0
fault_severity    0
severity_type     0
resource_type     0
log_feature       0
volume            0
event_type        0
dtype: int64

In [6]:
df.drop_duplicates(subset='id',keep='first',inplace=True)

In [7]:
df.head()

,id,location,fault_severity,severity_type,resource_type,log_feature,volume,event_type
0,14121,location 118,1,severity_type 2,resource_type 2,feature 312,19,event_type 34
4,9320,location 91,0,severity_type 2,resource_type 2,feature 315,200,event_type 34
8,14394,location 152,1,severity_type 2,resource_type 2,feature 221,1,event_type 35
12,8218,location 931,1,severity_type 1,resource_type 8,feature 80,9,event_type 15
18,14804,location 120,0,severity_type 1,resource_type 2,feature 134,1,event_type 34


In [8]:
df.shape

(7381, 8)

## Convert features into numbers

In [9]:
df.dtypes

id                 int64
location          object
fault_severity     int64
severity_type     object
resource_type     object
log_feature       object
volume             int64
event_type        object
dtype: object

In [10]:
def extract(column_names):
    for name in column_names:
        df[name] = df[name].str.extract(r'(\d+)').astype(int)
    return df
column_names = list(df.select_dtypes(include=['object']).columns)
df = extract(column_names)

In [11]:
df.dtypes

id                int64
location          int64
fault_severity    int64
severity_type     int64
resource_type     int64
log_feature       int64
volume            int64
event_type        int64
dtype: object

## Split dataset

In [12]:
y = df['fault_severity']
X = df.drop(['fault_severity','id'], axis=1)
print("Number of examples: " + str(X.shape[0]))
print("\nNumber of Features:" + str(X.shape[1]))
print(str(list(X.columns)))

Number of examples: 7381

Number of Features:6
['location', 'severity_type', 'resource_type', 'log_feature', 'volume', 'event_type']


## Logistic regression

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score


In [14]:
X_train,X_test, y_train,y_test =  train_test_split(X,y, test_size=0.33,random_state=1234)

In [15]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(4945, 6)
(4945,)
(2436, 6)
(2436,)


In [16]:
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr_model.fit(X_train, y_train)
lr_y_pred = lr_model.predict(X_test)

/Users/ghomefolder/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
from sklearn.metrics import accuracy_score, classification_report
LR_accuracy = accuracy_score(y_test, lr_y_pred)
LR_report = classification_report(y_test, lr_y_pred)
print('Accuracy score of model: ' + str(LR_accuracy))
print(LR_report)

Accuracy score of model: 0.6395730706075534
              precision    recall  f1-score   support

           0       0.65      0.97      0.78      1563
           1       0.40      0.06      0.10       591
           2       1.00      0.01      0.01       282

    accuracy                           0.64      2436
   macro avg       0.68      0.35      0.30      2436
weighted avg       0.63      0.64      0.52      2436



## KNN

In [18]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=3)

knn_model.fit(X_train, y_train) 

knn_y_pred = knn_model.predict(X_test) 

/Users/ghomefolder/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [19]:
KNN_accuracy = accuracy_score(y_test, knn_y_pred)
print('Accuracy score of model: ' + str(KNN_accuracy))

Accuracy score of model: 0.6342364532019704


## Decision Tree

In [20]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
# Create Decision Tree classifer object
dt_model = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train Decision Tree Classifer
dt_model = dt_model.fit(X_train,y_train)

#Predict the response for test dataset
dt_y_pred = dt_model.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, dt_y_pred))



Accuracy: 0.6543513957307061


## Random Forest

In [21]:
param_grid = {'max_depth':[4,8,10,20,32],'n_estimators':[25,50,100,200,300]}
print('Running grid search')
rf_model = RandomForestClassifier()
rf_grid = GridSearchCV(rf_model,param_grid,cv=3, scoring='neg_root_mean_squared_error')
rf_grid_search = rf_grid.fit(X_train,y_train)
print('finished grid search')

rf_best_params= rf_grid_search.best_params_
print(f"best parameters for rf model: {rf_best_params}")
rf_model = RandomForestClassifier(max_depth=rf_best_params['max_depth'],n_estimators = rf_best_params['n_estimators'])
rf_model.fit(X_train, y_train)
y_rf_pred=rf_model.predict(X_test)
   


Running grid search


NameError: name 'RandomForestClassifier' is not defined

In [ ]:
accuracy = accuracy_score(y_test, y_rf_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {'max_depth':[4,8,10],'n_estimators':[25,50,100,200,300]}
print('Running grid search')
gbdt_model = GradientBoostingClassifier()
GBDT_grid = GridSearchCV(gbdt_model,param_grid,cv=3, scoring='neg_root_mean_squared_error')
GBDT_grid_search = GBDT_grid.fit(X_train,y_train)
print('finished grid search')


GBDT_best_params= GBDT_grid_search.best_params_
print(f"best parameters for GBDT model: {GBDT_best_params}")

gbdt_model = GradientBoostingClassifier(max_depth=GBDT_best_params['max_depth'], n_estimators=GBDT_best_params['n_estimators'])
gbdt_model.fit(X_train,y_train)
y_gbdt_pred =  gbdt_model.predict(X_test)


In [ ]:
accuracy = accuracy_score(y_test, y_gbdt_pred)
print("Accuracy:", accuracy)